In [89]:
import pandas as pd
import pickle
import joblib
import numpy as np
import torch
from pytorch3d.transforms import quaternion_apply


In [71]:
def compute_joint_positions(joints_3d):
    joints_3d[:, [0, 1, 2]] = joints_3d[:, [0,2,1]]  # Make the 3rd value be the first on the second axis
    joints_3d[:, 2] *= -1  # Make the 3rd value negative
    joints_3d *= (0.34 / np.linalg.norm(joints_3d[3] - joints_3d[4]))
    joints_3d -= joints_3d[14]

    new_joints_3d = np.array([
        joints_3d[1],  # OP Neck (torso)
        joints_3d[0],  # OP Nose (head)
        joints_3d[8],  # OP MidHip (lower_waist)
        joints_3d[2],  # OP RShoulder (right_upper_arm)
        joints_3d[5],  # OP LShoulder (left_upper_arm)
        joints_3d[41],  # OP Spine (spine)
        joints_3d[3],  # OP RElbow (right_lower_arm)
        joints_3d[6],  # OP LElbow (left_lower_arm)
        joints_3d[9],  # OP RHip (right_thigh)
        joints_3d[12], # OP LHip (left_thigh)
        joints_3d[4],  # OP RWrist (right_hand)
        joints_3d[7],  # OP LWrist (left_hand)
        joints_3d[10], # OP RKnee (right_shin)
        joints_3d[13], # OP LKnee (left_shin)
        (0,0,0),          # Club (not available in JOINT_NAMES)
        joints_3d[11], # OP RAnkle (right_foot)
        joints_3d[14]  # OP LAnkle (left_foot)
    ])

    return new_joints_3d

In [126]:
import plotly.graph_objects as go
def visualize_joints(isaac_joints_3d):
    BONE_CONNECTIONS = {
        'neck_to_head': [0, 1],
        'neck_to_spine': [0, 5],
        'neck_to_right_upper_arm': [0, 3],
        'neck_to_left_upper_arm': [0, 4],
        'spine_to_lower_waist': [2, 5],
        'right_upper_arm_to_right_lower_arm': [3, 6],
        'left_upper_arm_to_left_lower_arm': [4, 7],
        'right_lower_arm_to_right_hand': [6, 10],
        'left_lower_arm_to_left_hand': [7, 11],
        'lower_waist_to_right_thigh': [2, 8],
        'lower_waist_to_left_thigh': [2, 9],
        'right_thigh_to_right_shin': [8, 12],
        'left_thigh_to_left_shin': [9, 13],
        'right_shin_to_right_foot': [12, 15],
        'left_shin_to_left_foot': [13, 16],  # Assuming Club is held in the right hand
    }

    # Extract the 3D joint coordinates
    joints_3d = isaac_joints_3d  # joints[0][:25]
    #joints_3d /= np.linalg.norm(joints_3d[3] - joints_3d[4])
    # Create a 3D scatter plot
    # print(joints_3d)
    # print(len(joints_3d))
    x_coords = [joint[0] for joint in joints_3d]
    y_coords = [joint[1] for joint in joints_3d]
    z_coords = [joint[2] for joint in joints_3d]
    # x_coords = [1, 2, 3]
    # y_coords = [4, 5, 6]
    # z_coords = [7, 8, 9]

    fig = go.Figure(data=[go.Scatter3d(
        x=x_coords,
        y=y_coords,
        z=z_coords,
        mode='markers',
        marker=dict(
            size=5,
            color=['blue' if i == 0 else 'red' for i in range(len(joints_3d))],  # Color the 45th joint in blue
        )
    )])

    # # Add lines for each bone defined in BONE_CONNECTIONS
    for bone_name, (start_joint, end_joint) in BONE_CONNECTIONS.items():
        fig.add_trace(go.Scatter3d(
            x=[joints_3d[start_joint][0], joints_3d[end_joint][0]],
            y=[joints_3d[start_joint][1], joints_3d[end_joint][1]],
            z=[joints_3d[start_joint][2], joints_3d[end_joint][2]],
            mode='lines',
            line=dict(color='blue', width=2),
            name=bone_name
        ))



    # Set labels
    fig.update_layout(
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z'
        )
    )

    # Show the plot
    fig.show()

In [147]:
def compute_quaternions(joints_3d):
    parent_joints = torch.stack([
        joints_3d[:, 0],  # neck (neck to head)
        joints_3d[:, 0],  # neck (neck to spine)
        joints_3d[:, 3],  # right_upper_arm (shoulder line)
        joints_3d[:, 2],  # spine (spine to lower waist)
        joints_3d[:, 3],  # right upper arm (right shoulder to elbow)
        joints_3d[:, 4],  # left upper arm (left shoulder to elbow)
        joints_3d[:, 6],  # right lower arm (right elbow to wrist)
        joints_3d[:, 7],  # left lower arm (left elbow to wrist)
        joints_3d[:, 8],  # right hip (waist line)
        joints_3d[:, 8],  # right hip (right_upper_leg)
        joints_3d[:, 9],  # left thigh (left_upper_leg)
        joints_3d[:, 12], # right shin (right_lower_leg)
        joints_3d[:, 13], # left shin (left_lower_leg)
        joints_3d[:, 15], # right foot (foot line)
    ], dim=1)
    child_joints = torch.stack([
        joints_3d[:, 1],  # head (neck to head)
        joints_3d[:, 5],  # spine (neck to spine)
        joints_3d[:, 4],  # left upper arm (shoulder line)
        joints_3d[:, 5],  # lower waist (spine to lower waist)
        joints_3d[:, 6],  # right lower arm (right shoulder to elbow)
        joints_3d[:, 7],  # left lower arm (left shoulder to elbow)
        joints_3d[:, 10], # right hand (right elbow to wrist)
        joints_3d[:, 11], # left hand (left elbow to wrist)
        joints_3d[:, 9],  # left hip (waist line)
        joints_3d[:, 12], # right shin (right upper leg)
        joints_3d[:, 13], # left shin (left upper leg)
        joints_3d[:, 15], # right foot (right lower leg)
        joints_3d[:, 16], # left foot (left lower leg)
        joints_3d[:, 16], # right foot (foot line)
    ], dim=1)

    bone_vectors = child_joints - parent_joints

    bone_vectors = bone_vectors / torch.norm(bone_vectors, dim=2, keepdim=True)

    # Define a reference vector
    ref_vecs = torch.zeros_like(bone_vectors)
    ref_vecs[:,:,2] = 1

    dot_product = torch.sum(bone_vectors * ref_vecs, dim=2, keepdim=True)
    cross_product = torch.cross(ref_vecs, bone_vectors, dim=2)
    
    w = torch.sqrt((torch.norm(bone_vectors, dim=2, keepdim=True) ** 2) * (torch.norm(ref_vecs, dim=2, keepdim=True) ** 2)) + dot_product
    quaternions = torch.cat((w, cross_product), dim=2)
    quaternions = quaternions / torch.norm(quaternions, dim=2, keepdim=True)
    print(bone_vectors)
    print(quaternion_apply(quaternions, ref_vecs))
    return quaternions

   

In [160]:
x = joblib.load('poses/results/demo_TIGER_WOODS_4HzLO88ryCU_a941d487-8693-40b6-8835-75b41c87b662.pkl')
quaternion_sequence = []
joint_positions = []
for i in range(1, 73):
    i_str = str(i).zfill(6)
    file_name = f'/home/daniel/development/csc2626/final-project/golfdb/data/poses//_DEMO/TIGER_WOODS_4HzLO88ryCU_a941d487-8693-40b6-8835-75b41c87b662/img/{i_str}.jpg'
    joints = x[file_name]['3d_joints']
    joints_3d = joints[0]

    new_joints_3d = compute_joint_positions(joints_3d)
    
    joint_positions.append(new_joints_3d.tolist())
    
    #visualize_joints(new_joints_3d)
    quaternions = compute_quaternions(torch.tensor([new_joints_3d]))
    quaternion_sequence.append(quaternions[0].tolist())
    joint_positions.append((new_joints_3d).tolist())
    # if i == 15:
    #     visualize_joints(new_joints_3d)
    #     break
    

import json

# Define the output file path
output_file_path = 'poses/results/quaternion_sequence2.json'

# Write the angle_sequence to the output file
with open(output_file_path, 'w') as f:
    json.dump(quaternion_sequence, f)

print(f"Angle sequence has been written to {output_file_path}")

with open('poses/results/joint_positions2.json', 'w') as f:
    json.dump(joint_positions, f)


tensor([[[ 0.9319,  0.0292, -0.3614],
         [-0.8527, -0.0150, -0.5222],
         [-0.0942,  0.9866,  0.1333],
         [ 0.4503, -0.0708,  0.8901],
         [-0.1086, -0.0190, -0.9939],
         [-0.1576,  0.1253, -0.9795],
         [ 0.0550,  0.3926, -0.9181],
         [ 0.1171, -0.2991, -0.9470],
         [-0.0757,  0.9933,  0.0878],
         [ 0.2989, -0.0715, -0.9516],
         [ 0.2833,  0.1933, -0.9394],
         [-0.0946,  0.0767, -0.9926],
         [-0.0294,  0.0019, -0.9996],
         [ 0.0408,  0.9941,  0.1000]]], dtype=torch.float64)
tensor([[[ 0.9319,  0.0292, -0.3614],
         [-0.8527, -0.0150, -0.5222],
         [-0.0942,  0.9866,  0.1333],
         [ 0.4503, -0.0708,  0.8901],
         [-0.1086, -0.0190, -0.9939],
         [-0.1576,  0.1253, -0.9795],
         [ 0.0550,  0.3926, -0.9181],
         [ 0.1171, -0.2991, -0.9470],
         [-0.0757,  0.9933,  0.0878],
         [ 0.2989, -0.0715, -0.9516],
         [ 0.2833,  0.1933, -0.9394],
         [-0.0946,  0.0767,

torch.Size([238, 13, 4])

In [ ]:
tensor([[[-0.0264, -0.8178,  0.0000,  0.5749],
         [ 0.0216,  0.8755, -0.0000,  0.4828],
         [ 0.6495,  0.0401, -0.0000,  0.7593],
         [-0.0474, -0.2270,  0.0000,  0.9727],
         [-0.1354,  0.9892,  0.0000,  0.0568],
         [ 0.7245,  0.6815, -0.0000,  0.1033],
         [ 0.9670, -0.1449,  0.0000,  0.2095],
         [-0.9525, -0.2577,  0.0000,  0.1624],
         [ 0.6677,  0.0475, -0.0000,  0.7429],
         [-0.2095, -0.9657,  0.0000,  0.1535],
         [ 0.6104, -0.7735,  0.0000,  0.1707],
         [ 0.6166,  0.7846, -0.0000,  0.0654],
         [ 0.5532,  0.8328, -0.0000,  0.0187]],

In [ ]:
tensor([[ 0.9403, -0.0304, -0.3391],
        [-0.8454,  0.0209, -0.5338],
        [-0.0609,  0.9863,  0.1532],
        [ 0.4415, -0.0923,  0.8925],
        [-0.1124, -0.0154, -0.9935],
        [-0.1408,  0.1497, -0.9786],
        [ 0.0607,  0.4052, -0.9122],
        [ 0.0837, -0.3093, -0.9473],
        [-0.0705,  0.9921,  0.1038],
        [ 0.2964, -0.0643, -0.9529],
        [ 0.2641,  0.2084, -0.9417],
        [-0.1025,  0.0806, -0.9915],
        [-0.0312,  0.0207, -0.9993]], dtype=torch.float64)

In [133]:
bone_vectors = torch.tensor([[[ 0.4954,  0.3277,  0.8045],
         [-0.3862, -0.2268, -0.8941],
         [-0.5338,  0.8455, -0.0157],
         [-0.2524, -0.1062, -0.9618],
         [ 0.2694, -0.4102, -0.8713],
         [-0.2361,  0.5046, -0.8304],
         [ 0.4411,  0.8963,  0.0452],
         [ 0.9852, -0.0236,  0.1700],
         [-0.4858,  0.8726,  0.0518],
         [-0.2913, -0.1056, -0.9508],
         [-0.0126, -0.1480, -0.9889],
         [-0.3001, -0.1348, -0.9443],
         [ 0.0093, -0.0938, -0.9956]]])

quaternions = torch.tensor([[[ 0.9499, -0.1725,  0.2608,  0.0000],
         [ 0.2301,  0.4928, -0.8392,  0.0000],
         [ 0.7015, -0.6026, -0.3805,  0.0000],
         [ 0.1382,  0.3840, -0.9129,  0.0000],
         [ 0.2537,  0.8086,  0.5309, -0.0000],
         [ 0.2912, -0.8665, -0.4054,  0.0000],
         [ 0.7229, -0.6199,  0.3051,  0.0000],
         [ 0.7649,  0.0154,  0.6440, -0.0000],
         [ 0.7252, -0.6016, -0.3349,  0.0000],
         [ 0.1569,  0.3367, -0.9285,  0.0000],
         [ 0.0745,  0.9936, -0.0848,  0.0000],
         [ 0.1668,  0.4041, -0.8994,  0.0000],
         [ 0.0472,  0.9940,  0.0982, -0.0000]]])

ref_vecs = torch.zeros_like(bone_vectors)
ref_vecs[:,:,2] = 1

print(bone_vectors)
print(quaternion_apply(quaternions, ref_vecs))


tensor([[[ 0.4954,  0.3277,  0.8045],
         [-0.3862, -0.2268, -0.8941],
         [-0.5338,  0.8455, -0.0157],
         [-0.2524, -0.1062, -0.9618],
         [ 0.2694, -0.4102, -0.8713],
         [-0.2361,  0.5046, -0.8304],
         [ 0.4411,  0.8963,  0.0452],
         [ 0.9852, -0.0236,  0.1700],
         [-0.4858,  0.8726,  0.0518],
         [-0.2913, -0.1056, -0.9508],
         [-0.0126, -0.1480, -0.9889],
         [-0.3001, -0.1348, -0.9443],
         [ 0.0093, -0.0938, -0.9956]]])
tensor([[[ 0.4955,  0.3277,  0.8045],
         [-0.3862, -0.2268, -0.8942],
         [-0.5338,  0.8454, -0.0158],
         [-0.2523, -0.1061, -0.9617],
         [ 0.2694, -0.4103, -0.8713],
         [-0.2361,  0.5046, -0.8304],
         [ 0.4411,  0.8963,  0.0452],
         [ 0.9852, -0.0236,  0.1701],
         [-0.4857,  0.8726,  0.0518],
         [-0.2914, -0.1057, -0.9509],
         [-0.0126, -0.1480, -0.9889],
         [-0.3000, -0.1348, -0.9444],
         [ 0.0093, -0.0938, -0.9955]]])
